In [1]:
import torch
from transformers import AutoTokenizer
from model_training import MultiLabelClassifier
import os

try:
    # 모델과 토크나이저 로드
    current_dir = os.path.dirname(os.path.abspath("__file__"))
    model_path = os.path.join(current_dir, "bert_model")
    
    # 모델 초기화
    model = MultiLabelClassifier(tokenizer_name="klue/bert-base")
    
    # 체크포인트 로드
    checkpoint_path = os.path.join(model_path, "best_model.pt")
    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"모델 파일을 찾을 수 없습니다: {checkpoint_path}")
    
    checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
    
    # 레이블 매핑 정의
    도수_매핑 = {0: '낮은', 1: '중간', 2: '높은'}
    술종류_매핑 = {0: '칵테일', 1: '럼', 2: '위스키', 3: '보드카'}
    맛_매핑 = {0: '달달한', 1: '쓴맛', 2: '상큼한', 3: '신맛', 4: '부드러운'}
    
    def predict(sentence):
        try:
            # 입력 문장 전처리
            inputs = tokenizer(
                sentence,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            )
            
            # 추론
            with torch.no_grad():
                outputs = model(
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask']
                )
            
            # 예측값 계산
            도수_pred = torch.argmax(outputs['도수'], dim=1).item()
            술종류_pred = torch.argmax(outputs['술종류'], dim=1).item()
            맛_pred = torch.argmax(outputs['맛'], dim=1).item()
            
            return {
                '도수': 도수_매핑[도수_pred],
                '술종류': 술종류_매핑[술종류_pred],
                '맛': 맛_매핑[맛_pred]
            }
        except Exception as e:
            print(f"예측 중 오류 발생: {str(e)}")
            return None

    # 테스트
    test_sentences = [
        "도수가 낮고 상큼한 칵테일 추천해줘.",
        "달달하고 도수 높은 럼 추천해줘.",
        "부드럽고 도수 중간인 위스키 추천해줘."
    ]

    for sentence in test_sentences:
        result = predict(sentence)
        if result:
            print(f"\n입력 문장: {sentence}")
            print(f"예측 결과:")
            print(f"- 도수: {result['도수']}")
            print(f"- 술종류: {result['술종류']}")
            print(f"- 맛: {result['맛']}")

except Exception as e:
    print(f"초기화 중 오류 발생: {str(e)}")

d:\project\OSS_Project\AI\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


초기화 중 오류 발생: name '__file__' is not defined
